In [1]:
#-----------------------------------------------------------------------
# 1. CONFIGURAÇÃO INICIAL E PARÂMETROS
#-----------------------------------------------------------------------

# Instalar pacotes necessários (se ainda não tiver)
# pip install yfinance pandas numpy scipy

import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import date, timedelta

# Parâmetros dos Ativos e do Mercado
tickers = ["PETR4.SA", "VALE3.SA"]
weights = np.array([0.5, 0.5])

# Parâmetros da Opção
K = 40.0      # Preço de exercício (Strike)
T = 1/12      # Maturidade em anos (1 mês)

# Taxa de juros livre de risco (Selic). Usaremos uma taxa aproximada de 10.5% a.a.
# É importante usar uma taxa atual para maior precisão.
risk_free_rate = 0.105 

# Parâmetros da Simulação de Monte Carlo
n_simulations = 100000 # Número de simulações (quanto maior, mais preciso)
np.random.seed(123)    # Para reprodutibilidade dos resultados

#-----------------------------------------------------------------------
# 2. COLETA E PREPARAÇÃO DE DADOS
#-----------------------------------------------------------------------
print("Baixando dados históricos...")

end_date = date.today()
start_date = end_date - timedelta(days=365)

# Baixar os dados usando a biblioteca yfinance
try:
    prices = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    prices.dropna(inplace=True) # Remover dias em que um dos ativos não foi negociado
except Exception as e:
    print(f"Erro ao baixar os dados: {e}")
    exit()

# Calcular os retornos logarítmicos diários
log_returns = np.log(prices / prices.shift(1))
log_returns.dropna(inplace=True)

#-----------------------------------------------------------------------
# 3. CALIBRAÇÃO (Cálculo dos parâmetros estatísticos)
#-----------------------------------------------------------------------
print("Calibrando os parâmetros a partir dos dados...\n")

# Volatilidades anuais (desvio padrão dos retornos logarítmicos diários * sqrt(252))
trading_days = 252
vol_annual = log_returns.std() * np.sqrt(trading_days)

# Matriz de correlação dos retornos
correlation_matrix = log_returns.corr()
correlation = correlation_matrix.iloc[0, 1]

# Preços iniciais (último preço de fechamento disponível)
initial_prices = prices.iloc[-1].values

# Imprimir parâmetros calculados
print("--- Parâmetros Calibrados ---")
print(f"Preço Inicial PETR4 (S0): {initial_prices[0]:.2f}")
print(f"Preço Inicial VALE3 (S0): {initial_prices[1]:.2f}")
print(f"Volatilidade Anualizada PETR4: {vol_annual[0]:.2%}")
print(f"Volatilidade Anualizada VALE3: {vol_annual[1]:.2%}")
print(f"Correlação entre PETR4 e VALE3: {correlation:.4f}")
print("-----------------------------\n")

#-----------------------------------------------------------------------
# 4. PRECIFICAÇÃO VIA SIMULAÇÃO DE MONTE CARLO
#-----------------------------------------------------------------------
print("Iniciando a simulação de Monte Carlo...")

# Gerar números aleatórios normais padrão (independentes)
Z = np.random.normal(size=(n_simulations, len(tickers)))

# Aplicar a decomposição de Cholesky para correlacionar os números aleatórios
# L é a matriz triangular inferior da decomposição
L = np.linalg.cholesky(correlation_matrix)
Z_correlated = Z @ L.T

# Simular os preços no vencimento (ST) para cada ativo
# Usamos a fórmula do Movimento Browniano Geométrico sob a medida risco-neutro
# O drift (deriva) é a taxa livre de risco 'r'
drift = (risk_free_rate - 0.5 * vol_annual.values**2) * T
diffusion = vol_annual.values * np.sqrt(T) * Z_correlated

ST = initial_prices * np.exp(drift + diffusion)

# Calcular o valor da cesta no vencimento para cada simulação
ST_basket = np.sum(ST * weights, axis=1)

# Calcular o payoff da call para cada simulação: max(ST_cesta - K, 0)
payoffs = np.maximum(ST_basket - K, 0)

# O preço da opção é a média dos payoffs descontada a valor presente
monte_carlo_price = np.mean(payoffs) * np.exp(-risk_free_rate * T)

#-----------------------------------------------------------------------
# 5. PRECIFICAÇÃO VIA BLACK-SCHOLES (APROXIMAÇÃO PARA A CESTA)
#-----------------------------------------------------------------------
print("Calculando o preço via fórmula de Black-Scholes para a cesta...")

# Preço inicial da cesta
S0_basket = np.sum(initial_prices * weights)

# Calcular a volatilidade da cesta
# A variância da cesta é: w1^2*sd1^2 + w2^2*sd2^2 + 2*w1*w2*sd1*sd2*corr
var_basket = (weights[0]**2 * vol_annual[0]**2) + \
             (weights[1]**2 * vol_annual[1]**2) + \
             (2 * weights[0] * weights[1] * vol_annual[0] * vol_annual[1] * correlation)
vol_basket = np.sqrt(var_basket)

# Função para a fórmula de Black-Scholes
def black_scholes_call(S, K, T, r, sigma):
  d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
  return price

# Calcular o preço usando os parâmetros da cesta
black_scholes_price = black_scholes_call(S0_basket, K, T, risk_free_rate, vol_basket)

#-----------------------------------------------------------------------
# 6. RESULTADOS
#-----------------------------------------------------------------------
print("\n--- RESULTADOS DA PRECIFICAÇÃO ---")
print(f"Preço da Opção (Monte Carlo): R$ {monte_carlo_price:.4f}")
print(f"Preço da Opção (Black-Scholes para Cesta): R$ {black_scholes_price:.4f}")
print("------------------------------------")

Failed to get ticker 'VALE3.SA' reason: Expecting value: line 1 column 1 (char 0)


Baixando dados históricos...
[*********************100%%**********************]  2 of 2 completed


2 Failed downloads:
['VALE3.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')
['PETR4.SA']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-08-27 -> 2025-08-27)')



Calibrando os parâmetros a partir dos dados...



IndexError: single positional indexer is out-of-bounds